In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [ ]:
os.chdir('/content/drive/MyDrive/리스크공모전')
os.getcwd()

'/content/drive/MyDrive/리스크공모전'

In [ ]:
peach=pd.read_csv('복숭아 개화 데이터.csv', encoding='cp949')
pair=pd.read_csv('배 개화 데이터.csv', encoding='cp949')

In [ ]:
peach.drop(['index'], axis=1, inplace=True)
pair.drop(['index'], axis=1, inplace=True)

In [ ]:
peach['지점'].unique()

array(['강릉', '대관령', '동해', '북강릉', '북춘천', '삼척', '속초', '영월', '원주', '인제',
       '철원', '춘천', '태백', '홍천', '동두천', '수원', '양평', '이천', '파주', '거제', '거창',
       '남해', '밀양', '산청', '진주', '창원', '통영', '합천', '구미', '문경', '봉화', '상주',
       '안동', '영덕', '영주', '영천', '울릉도', '울진', '의성', '포항', '광주', '대구',
       '대구(기)', '대전', '부산', '서울', '울산', '강화', '백령도', '인천', '고흥', '목포',
       '무안', '순천', '여수', '완도', '장흥', '주암', '진도(첨찰산)', '해남', '흑산도', '고창',
       '고창군', '군산', '남원', '부안', '임실', '장수', '전주', '정읍', '고산', '서귀포', '성산',
       '성산포', '제주', '금산', '보령', '부여', '서산', '천안', '홍성', '보은', '제천', '청주',
       '추풍령', '충주'], dtype=object)

In [ ]:
peach.columns

Index(['지점', '연', '복숭아 개화', '복숭아 개화(평비)', '365', '서리끝', '서리끝(평비)', '서리365',
       '위도', '경도', '고도', '해양', '2_일강수량(mm)_sum', '3_일강수량(mm)_sum',
       '2_최고기온(°C)_max', '3_최고기온(°C)_max', '2_최고기온(°C)_mean',
       '3_최고기온(°C)_mean', '2_최고기온(°C)_min', '3_최고기온(°C)_min', '2_최저기온(°C)_max',
       '3_최저기온(°C)_max', '2_최저기온(°C)_mean', '3_최저기온(°C)_mean',
       '2_최저기온(°C)_min', '3_최저기온(°C)_min', '2_평균기온(°C)_max', '3_평균기온(°C)_max',
       '2_평균기온(°C)_mean', '3_평균기온(°C)_mean', '2_평균기온(°C)_min',
       '3_평균기온(°C)_min', '2_합계 일조시간(hr)_sum', '3_합계 일조시간(hr)_sum'],
      dtype='object')

In [ ]:
#train, test set 지점별로 0.8:0.2
def traintest_split(data):
    unique_locations=data['지점'].unique()
    train=pd.DataFrame()
    test=pd.DataFrame()

    for location in tqdm(unique_locations, desc='Processing Locations'):
        location_data=data[data['지점']==location]
        total_rows=len(location_data)
        train_rows=int(total_rows*0.8)
        tr_data=location_data.iloc[:train_rows]
        te_data=location_data.iloc[train_rows:]

        train=pd.concat([train, tr_data])
        test=pd.concat([test, te_data])
    return train, test

train_peach, test_peach = traintest_split(peach)
train_pair, test_pair = traintest_split(pair)

Processing Locations: 100%|██████████| 86/86 [00:00<00:00, 172.04it/s]


In [ ]:
train_peach.drop(['지점','연','복숭아 개화','복숭아 개화(평비)','서리끝'],axis=1,inplace=True)
test_peach.drop(['지점','연','복숭아 개화','복숭아 개화(평비)','서리끝'],axis=1,inplace=True)
train_pair.drop(['지점','연','배나무 개화','배나무 개화(평비)','서리끝'],axis=1,inplace=True)
test_pair.drop(['지점','연','배나무 개화','배나무 개화(평비)','서리끝'],axis=1,inplace=True)

#RandomForest

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

In [ ]:
x_train_peach=train_peach.drop(['365'],axis=1)
y_train_peach=train_peach['365']
x_test_peach=test_peach.drop(['365'],axis=1)
y_test_peach=test_peach['365']

In [ ]:
model = RandomForestRegressor()
model.fit(x_train_peach, y_train_peach)
y_predict_peach = model.predict(x_test_peach)
y_predict_peach=np.round(y_predict_peach)

In [ ]:
mean_absolute_error(y_test_peach, y_predict_peach)

3.68

In [ ]:
# Define objective function
def objective(trial):
    # Suggest values for hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 10, 200, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Create and fit random forest model
    model = RandomForestRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    random_state=42,
    )
    model.fit(x_train_peach, y_train_peach)

    # Make predictions and calculate RMSE
    y_pred_peach = model.predict(x_test_peach)
    rmse = np.sqrt(mean_squared_error(y_test_peach, y_pred_peach))
    mae = mean_absolute_error(y_test_peach, y_pred_peach)
    r2 = r2_score(y_test_peach, y_pred_peach)

    # Return MAE
    return mae

In [ ]:
# Create study object
study = optuna.create_study(direction="minimize")

# Run optimization process
study.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2023-10-02 02:32:20,009] A new study created in memory with name: no-name-73b40b6a-d963-43b7-a4e3-21f250bc6fdf


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2023-10-02 02:32:21,712] Trial 0 finished with value: 3.7264626739386 and parameters: {'n_estimators': 103, 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 0 with value: 3.7264626739386.
[I 2023-10-02 02:32:22,698] Trial 1 finished with value: 4.198392991372715 and parameters: {'n_estimators': 174, 'max_depth': 2, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 0 with value: 3.7264626739386.
[I 2023-10-02 02:32:23,204] Trial 2 finished with value: 3.74509757041516 and parameters: {'n_estimators': 33, 'max_depth': 8, 'min_samples_split': 10, 'min_samples_leaf': 8}. Best is trial 0 with value: 3.7264626739386.
[I 2023-10-02 02:32:23,417] Trial 3 finished with value: 3.877608441441384 and parameters: {'n_estimators': 10, 'max_depth': 24, 'min_samples_split': 9, 'min_samples_leaf': 5}. Best is trial 0 with value: 3.7264626739386.
[I 2023-10-02 02:32:25,949] Trial 4 finished with value: 3.703779700910201 and parameters: {'n_estimators': 125, 'm

In [ ]:
# Print best trial and best hyperparameters
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=13, state=TrialState.COMPLETE, values=[3.657083786693154], datetime_start=datetime.datetime(2023, 10, 2, 2, 32, 42, 378273), datetime_complete=datetime.datetime(2023, 10, 2, 2, 32, 45, 79404), params={'n_estimators': 87, 'max_depth': 30, 'min_samples_split': 7, 'min_samples_leaf': 1}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=200, log=True, low=10, step=1), 'max_depth': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=10, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=10, log=False, low=1, step=1)}, trial_id=13, value=None)
Best hyperparameters: {'n_estimators': 87, 'max_depth': 30, 'min_samples_split': 7, 'min_samples_leaf': 1}


In [ ]:
model = RandomForestRegressor(**study.best_params)
model.fit(x_train_peach, y_train_peach)
y_pred_peach = model.predict(x_test_peach)
mae = mean_absolute_error(y_test_peach, y_pred_peach)
mae

3.676880115729738

#Xgboost

In [ ]:
from sklearn.preprocessing import StandardScaler
import xgboost

In [ ]:
# Scale the data
scaler = StandardScaler()
x_train_peach = scaler.fit_transform(x_train_peach)
x_test_peach = scaler.transform(x_test_peach)

In [ ]:
model = xgboost.XGBRegressor()
model.fit(x_train_peach, y_train_peach)
y_predict_peach = model.predict(x_test_peach)
y_predict_peach=np.round(y_predict_peach)

In [ ]:
mean_absolute_error(y_test_peach, y_predict_peach)

3.92

In [ ]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = xgboost.XGBRegressor(**param)
    model.fit(x_train_peach, y_train_peach)
    y_pred_peach = model.predict(x_test_peach)
    return mean_squared_error(y_test_peach, y_pred_peach)

In [ ]:
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=100)

[I 2023-10-02 02:33:58,434] A new study created in memory with name: regression
[I 2023-10-02 02:33:59,000] Trial 0 finished with value: 2.703888945321654e+38 and parameters: {'max_depth': 9, 'learning_rate': 0.9365127479569196, 'n_estimators': 955, 'min_child_weight': 5, 'gamma': 0.25297898280265807, 'subsample': 0.045531428430309565, 'colsample_bytree': 0.26303831190413485, 'reg_alpha': 0.5613711229658509, 'reg_lambda': 0.06559886072192428, 'random_state': 23}. Best is trial 0 with value: 2.703888945321654e+38.
[I 2023-10-02 02:33:59,701] Trial 1 finished with value: 40.818798939242264 and parameters: {'max_depth': 3, 'learning_rate': 0.31623550726755734, 'n_estimators': 423, 'min_child_weight': 4, 'gamma': 0.2613278266229911, 'subsample': 0.4497803009613127, 'colsample_bytree': 0.5750184042905974, 'reg_alpha': 0.85746808625259, 'reg_lambda': 0.47871569952869186, 'random_state': 635}. Best is trial 1 with value: 40.818798939242264.
[I 2023-10-02 02:34:02,317] Trial 2 finished with va

In [ ]:
# Print the best parameters
print('Best parameters', study.best_params)

Best parameters {'max_depth': 9, 'learning_rate': 0.06144858368592065, 'n_estimators': 895, 'min_child_weight': 6, 'gamma': 0.28210253499240096, 'subsample': 0.6807677063742514, 'colsample_bytree': 0.28157670018019343, 'reg_alpha': 0.5484572029127113, 'reg_lambda': 0.44039538103519243, 'random_state': 403}


In [ ]:
model = xgboost.XGBRegressor(**study.best_params)
model.fit(x_train_peach, y_train_peach)
y_pred_peach = model.predict(x_test_peach)

print('MSE: ', mean_squared_error(y_test_peach, y_pred_peach))
print('RMSE: ', np.sqrt(mean_squared_error(y_test_peach, y_pred_peach)))

MSE:  26.862968380123494
RMSE:  5.182949775959969


In [ ]:
mean_absolute_error(y_test_peach, y_pred_peach)

3.6467140197753904

# LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

model=LinearRegression()
model.fit(x_train_peach, y_train_peach)
y_predict_peach=model.predict(x_test_peach)

(400, 28)

# RidgeRegression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

alphas = np.arange(1, 50, 1)

model1 = RidgeCV(alphas=alphas, cv=2)
model1.fit(x_train_peach, y_train_peach)
y_predict_peach = model1.predict(x_test_peach)

In [ ]:
print('회귀계수:', model1.coef_[0])
print('최적알파:', model1.alpha_)

회귀계수: 0.20010798301713673
최적알파: 49
